In [1]:
import time
import logging
import numpy as np
import matplotlib.pyplot as plt
plt.ion()

import qcodes as qc
from qcodes.instrument_drivers.ZI.ZIUHFLI import ZIUHFLI

log = logging.getLogger(__name__)
log.setLevel(logging.INFO)

# Prerequisites

It is necessary to download and install the ZI Lab One software. Additionally, both the data server and the web server must run, and a connection to the instrument must be instantiated (this can be done via the web interface).

In [2]:
# Instantiate the QCoDeS instrument
zi = ZIUHFLI('ZIUHFLI', 'dev2235')

# Basic Usage of the ZI UHF-LI

Most of the "front panel" (i.e. the Web UI) `Lock-In` and `signal input` settings are available as parameters.

In [ ]:
zi.oscillator1_freq.set(752.1e3)
print('Oscillator 2 has frequency: {:.0f} Hz'.format(zi.oscillator2_freq.get()))
zi.signal_input1_range(1)
zi.signal_input1_scaling(1)

Each demodulator has several settings...

In [ ]:
print('Available demodulator settings:\n')
for param in [p for p in zi.parameters if 'demod1' in p]:
    print('  {}, {} ({})'.format(param, zi.parameters[param].label, zi.parameters[param].unit))

# Using the sweeper

The sweeper settings are configured via a bunch of parameters, all named `sweeper_XXX`.
This configures the x-axis of the sweep as well as the sweep acquisition settings. To learn more about what a certain parameter does, it is sometimes helpful to print its `__doc__` attribute.

Which signals are **returned** by the sweeper is controlled by adding (removing) signals to (from) the sweep.

The sweep settings can be displayed with the `print_sweeper_settings` command.

Before the sweep can be performed, it must be built. 
This is done with the Sweep parameter, which is the parameter holding the sweep data.
Note that building the sweep may change some of the time constants, and in particular change the sweep time. In case of doubt, re-run `print_sweeper_settings`.

In [ ]:
# Set up a sweep sweeping an internal oscillator frequency from 1 MHz to 5 MHz
# We want the sweeper to sweep linearly over 200 points
zi.sweeper_param('Osc 1 Frequency')
zi.sweeper_xmapping('lin')
zi.sweeper_start(1e6)
zi.sweeper_stop(10e6)
zi.sweeper_samplecount(100)
zi.sweeper_BWmode('fixed')
zi.sweeper_BW(250)
zi.sweeper_order(4)

In [ ]:
# I wonder what the sweeper BWmode does...
print(zi.sweeper_BWmode.__doc__)

In [ ]:
# Add three signals to the sweep, all measured on demodulator 1
zi.add_signal_to_sweeper(1, 'Xrms')
zi.add_signal_to_sweeper(1, 'Yrms')
zi.add_signal_to_sweeper(1, 'Rrms')

In [ ]:
# Make sure that demodulator 1 is measuring what and as it should
zi.demod1_trigger('Continuous')
zi.demod1_signalin('Sig In 1')

In [ ]:
# I wonder what kind of sweep we have made now...
zi.print_sweeper_settings()

In [ ]:
# Gee, that looks good! Note the last line, the sweep is NOT ready to execute.
zi.Sweep.build_sweep()
# Now it is!
zi.print_sweeper_settings()

In [ ]:
# We can now execute the sweeper by simply invoking Sweep.get
# This returns a tuple with the signals we asked for
(X, Y, R) = zi.Sweep.get()

In [ ]:
# This we may manually plot. 

In [ ]:
plt.figure()
plt.plot(X, lw=2, color=(0.5, 0, 0.3))
plt.plot(Y, lw=2, color=(0.5, 0.3, 0))
plt.plot(R, lw=2, color=(0.3, 0, 0.5))

In [ ]:
# zi.close()

In [ ]:
print(zi.daq.get('/dev2235/sigouts/0/amplitudes/3'))

In [ ]:
zi.list_nodes("sigouts/*/amplitudes")

In [ ]:
zi._list_nodes("sigouts/*")

In [4]:
import qcodes.utils.validators as vals
from qcodes.instrument.parameter import ManualParameter

In [5]:
bob = ManualParameter(name='testparam', vals=vals.Numbers(1, 2.5))

In [9]:
zi.oscillator1_freq.get()

420000.0000011528

In [10]:
zi.oscillator1_freq.get_latest()

420000.0000011528

In [ ]:
bob.get_latest()